In [2]:
pip install selenium

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ------- -------------------------------- 1.8/9.7 MB 11.2 MB/s eta 0:00:01
   -------------- ------------------------- 3.4/9.7 MB 9.2 MB/s eta 0:00:01
   ---------------------- ----------------- 5.5/9.7 MB 9.6 MB/s eta 0:00:01
   ------------------------------ --------- 7.3/9.7 MB 9.3 MB/s eta 0:00:01
   ------------------------------------- -- 9.2/9.7 MB 9.5 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 9.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ------- -------------------------------- 2.1/11.6 MB 11.8 MB/s eta 0:00:01
   -------------- ------------------------- 4.2/11.6 MB 11.0 MB/s eta 0:00:01
   --------------------- ------------------ 6.3/11.6 MB 10.7 MB/s eta 0:00:01
   ----------------------------- ---------- 8.7/11.6 MB 10.7 MB/s eta 0:00:01
   ------------------------------------- -- 10.7/11.6 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 10.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.9 MB 11.8 MB/s eta 0:00:01
   ------------ --------------------------- 4.2/12.9 MB 11.0 MB/s eta 0:00:01
   ----------------- ---------------------- 5.8/12.9 MB 9.8 MB/s eta 0:00:01
   ----------------------- ---------------- 7.6/12.9 MB 9.6 MB/s eta 0:00:01
   ----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd

###Chrome ####
c_options = webdriver.ChromeOptions()
c_options.add_argument("--start-maximized")
c_driver = webdriver.Chrome(options=c_options) 

In [80]:
c_driver.get("https://www.tesla.com/es_ES/inventory/new/my?arrangeby=plh&zip=28008&range=0")
time.sleep(2)

In [ ]:
#botton_cookies = //*[@id="tsla-accept-cookie"]
botton_cookies = c_driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/button[1]')
botton_cookies.click()
time.sleep(2)



In [82]:
def obtener_articulos_pagina():
    articulos = []
    try:

        articulos = c_driver.find_elements(By.TAG_NAME, 'article')
        #print(articulos)
        
        return articulos
    except NoSuchElementException:
        print("No se encontraron elementos con la clase especificada")

In [ ]:
def generar_diccionario_articulo(articulo, estado_coche):
    diccionario_articulo = {}
    datos_generales = articulo.find_element(By.CLASS_NAME, 'result-basic-info')
    elementos_span_result_basic_info = datos_generales.find_elements(By.TAG_NAME, 'span')

    
    diccionario_articulo['modelo'] = elementos_span_result_basic_info[0].text
    try:
        diccionario_articulo['km'] = elementos_span_result_basic_info[1].text.split("con")[1]
    except IndexError:
        diccionario_articulo['km'] = 0

    try:
        precio = articulo.find_element(By.CLASS_NAME, 'result-pricing')   
        
        valor_precio = precio.find_element(By.TAG_NAME, 'span')
        diccionario_articulo['precio'] = valor_precio.text  
    except NoSuchElementException:
        diccionario_articulo['precio'] = 0
    try:
        div_pintura = articulo.find_elements(By.CLASS_NAME, 'tds-list-item')
        
        diccionario_articulo['pintura'] = div_pintura[2].text
    except NoSuchElementException:
        pintura = "No hay pintura"
        diccionario_articulo['pintura'] = pintura
    diccionario_articulo['estado'] = estado_coche
    
    #print(diccionario_articulo)
def almacenar_en_lista(lista_diccionarios_datos_articulos, articulos, estado_coche):
    for articulo in articulos:
        diccionario = generar_diccionario_articulo(articulo, estado_coche)
        lista_diccionarios_datos_articulos.append(diccionario)
    
    

In [ ]:
lista_diccionarios_datos_articulos = []
botones_filtros = []
div_botones_modelos = c_driver.find_element(By.CLASS_NAME, 'tds-form-input-group')
elementos_botones_modelos = div_botones_modelos[0].find_elements(By.TAG_NAME, 'input')
div_tipo_inventario = c_driver.find_element(By.CLASS_NAME, 'tds-form-item')
elementos_botones_inventario = div_tipo_inventario[0].find_elements(By.TAG_NAME, 'input')
for modelo in elementos_botones_modelos:
    for inventario in elementos_botones_inventario:
        valor_inventario = inventario.get_attribute('value')
        modelo.click()
        inventario.click()
        #hacemos scroll hasta abajo para cargar todos los elementos
        c_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        articulos = obtener_articulos_pagina()
        almacenar_en_lista(lista_diccionarios_datos_articulos, articulos, valor_inventario)

        


"""articulos = obtener_articulos_pagina()
almacenar_en_lista(lista_diccionarios_datos_articulos, articulos)"""


    


'articulos = obtener_articulos_pagina()\nalmacenar_en_lista(lista_diccionarios_datos_articulos, articulos)'